In [1]:
import cv2
import sys
import numpy as np

## This n. network can detect digits "1" anf "2" by photos

## Code below prepare images for dataset

In [2]:
#for i in range(30):    
#    img = cv2.imread('images/data_{}.jpg'.format(i), cv2.IMREAD_GRAYSCALE)
#    res_img = cv2.resize(img, (28, 28), cv2.INTER_NEAREST)
    #cv2.imshow('number', res_img)
    #cv2.waitKey(0)
#    cv2.imwrite('rgimages/data_{}_gray.jpg'.format(i), res_img)

## Neural network itself

In [2]:
def neural_network(inputs, weights):
    
    #output vector size
    n = np.shape(weights)[0]
    
    #make output vector
    ans = np.zeros(n)
    
    # computing prediction
    ans = np.dot(weights, inputs)
    
    return ans

In [3]:
def img_to_vect(img): #function that transforms matrix to vector
    
    #obtain size of vector
    h = np.shape(img)[0]
    w = np.shape(img)[1]
    vect = np.zeros(h * w)
    
    #overwrite image to vector
    for i in range(h):
        for j in range(w):
            vect[i * w + j] = img[i][j]
    
    return vect

In [4]:
def vect_to_img(vect, h, w):#function that transforms vector to matrix
    
    #size correctness verifying
    assert(h * w == len(vect))
    img = np.zeros((h, w))
    
    #overwrite vector to matrix
    for i in range(len(vect)):
        k = i // h
        j = i % w
        img[k][j] = vect[k * w + j]
        
    return img

## Create weights, which are arbitary

In [104]:
wimg = np.ones((2, 28, 28))
#cv2.imwrite('weights_number2.jpg', weights)
weights = [img_to_vect(wimg[i]) for i in range(2)]

## Learning block

In [132]:
#alpha-cofficient
alpha = 0.001

for i in range(60):
    
    #upload image
    img = cv2.imread('rgimages/data_{}_gray.jpg'.format(i % 30), cv2.IMREAD_GRAYSCALE)
    
    #upload what is written on image '1' or '2' 
    with open("ans/out_{}.txt".format(i % 30), "r") as f: 
        data_o = f.read(2)
        true = np.zeros(len(data_o))
        for j in range(2):
            true[j] = (data_o[j] == '1') * 1.0
    
    #convert image to vector
    data_i = img_to_vect(img) / 255.0
    
    #make prediction
    pred = neural_network(data_i, weights)

    #pure mistake
    delta = pred - true
    
    #sqare error
    err = np.dot(delta, delta)
    
    #correct weights using current prediction
    for k in range(2):
        weights_deltas = data_i * delta[k]
        weights[k] -= alpha * weights_deltas
    print("iter: {} error: {} pred: 1 => {}, 2 => {}".format(i, round(err, 4), round(pred[0], 4), round(pred[1], 4)))


#save weights

iter: 0 error: 0.0854 pred: 1 => 0.708, 2 => -0.0106
iter: 1 error: 0.0779 pred: 1 => 0.882, 2 => -0.253
iter: 2 error: 0.0156 pred: 1 => -0.046, 2 => 0.884
iter: 3 error: 0.0298 pred: 1 => -0.0441, 2 => 0.8331
iter: 4 error: 0.0246 pred: 1 => 0.9657, 2 => -0.153
iter: 5 error: 0.0275 pred: 1 => 0.1355, 2 => 0.9045
iter: 6 error: 0.0097 pred: 1 => 1.0929, 2 => 0.033
iter: 7 error: 0.0526 pred: 1 => 0.1851, 2 => 0.8646
iter: 8 error: 0.0012 pred: 1 => 0.996, 2 => -0.0345
iter: 9 error: 0.0175 pred: 1 => 0.0803, 2 => 0.8949
iter: 10 error: 0.0012 pred: 1 => 0.0307, 2 => 1.0165
iter: 11 error: 0.0018 pred: 1 => 0.0282, 2 => 0.9679
iter: 12 error: 0.1198 pred: 1 => 0.342, 2 => 0.9466
iter: 13 error: 0.021 pred: 1 => 0.1311, 2 => 0.9387
iter: 14 error: 0.0216 pred: 1 => 0.1356, 2 => 0.9431
iter: 15 error: 0.1621 pred: 1 => 0.8536, 2 => 0.3751
iter: 16 error: 0.0248 pred: 1 => 0.1246, 2 => 0.9035
iter: 17 error: 0.0051 pred: 1 => 1.0015, 2 => 0.0715
iter: 18 error: 0.0312 pred: 1 => 0.1766, 

## Code below presents weights as image: the heavier the weight, the lighter the corresponding item

In [131]:
for i in range(2):
    
    #convert weights to pixel-values range
    wimg[i] = vect_to_img(weights[i], 28, 28)
    c = max(weights[i]) - min(weights[i])
    n_weights = (255 * ((weights[i] - min(weights[i]) * np.ones(len(weights[i])) / c)))
    
    #convert weights to matrix
    wimg[i] = vect_to_img(n_weights, 28, 28)
    wimg[i] = wimg[i].astype(int)
    
    #record image
    cv2.imwrite('weights_number{}.jpg'.format(i + 1), wimg[i])


In [ ]:
#convert test image to data format 
tst = cv2.imread('images/k-test.jpg', cv2.IMREAD_GRAYSCALE)
res_tst = cv2.resize(tst, (28, 28), cv2.INTER_NEAREST)
cv2.imwrite('rgimages/k-test_gray.jpg'.format(i), res_tst)

In [130]:
tst_img = cv2.imread('rgimages/k-test_gray.jpg', cv2.IMREAD_GRAYSCALE)
data_test = img_to_vect(tst_img) / 255.0

#make prediction
pred = neural_network(data_test, weights)
delta = pred - np.array([0.0, 1.0])
err = np.dot(delta, delta)
print("error: {} pred: 1 => {} 2 => {}".format(round(err, 4), round(pred[0], 4), round(pred[1], 4)))

error: 0.1202 pred: 1 => 0.1665 2 => 0.6959
